In [70]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
import numpy as np
import pandas as pd
import string
import re
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BertJapaneseTokenizer, BertModel
from torch import cuda
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import logging


In [72]:
batch_size = 16
max_len = 256

In [73]:
df_train = pd.read_csv("./drive/MyDrive/Colab_Notebooks/data/IMDb/IMDb_train.tsv", sep="\t", header=None)
df_train = df_train.iloc[:, 0:2]
df_train.columns = ["text", "label"]
print(df_train.shape)
df_train.head()

(25000, 2)


,text,label
0,The Unborn is a pretty good low-budget horror ...,1
1,Vincente Minnelli directed some of the most ce...,1
2,"The first time I saw this, I didn't laugh too ...",1
3,This is a great movie for all Generation X'ers...,1
4,I first saw this absolutely riveting documenta...,1


In [74]:
df_test = pd.read_csv("./drive/MyDrive/Colab_Notebooks/data/IMDb/IMDb_test.tsv", sep="\t", header=None)
df_test = df_test.iloc[:, 0:2]
df_test.columns = ["text", "label"]
print(df_test.shape)
df_test.head()

(25000, 2)


,text,label
0,"Susan Sarandon is, for lack of a better word, ...",1
1,Seeing Laurel without Hardy in a film seems st...,1
2,I was recently at a sleepover birthday party w...,1
3,This movie took me by surprise. The opening cr...,1
4,A widely unknown strange little western with m...,1


In [75]:
df_train["label"].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [76]:
df_train["text"].map(len).median()

979.0

In [77]:
df_train["text"].map(len)

0         639
1        4018
2         921
3        1582
4        1526
         ... 
24995     472
24996     751
24997     964
24998    6041
24999    1405
Name: text, Length: 25000, dtype: int64

# 前処理

In [78]:
def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text


In [79]:
df_train["text"].map(preprocessing_text)
preprocessing_text(df_train["text"][0])

'The Unborn is a pretty good low budget horror movie exploiting the fears associated with pregnancy .  It s very well acted by the always good Brooke Adams and b movie stalwart James Karen ,  although the supporting cast is pretty average for a b grader .  The music ,  by Gary Numan of all people ,  is good too .  Henry Dominic s script is quite intelligent for this sort of thing ,  although there is a hint of misogyny about it .  Rodman Fender s direction is merely adequate ,  and there are some unnecessary cheap scares .  If you re a fan of Adams ,  whose movie career is nowhere near as illustrious as it should be ,  check it out  she s great ,  as always . '

# データセット作成

In [80]:
class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer, max_len):
    self.X = X
    self.y = y
    # self.uid = uid
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.y)

  def encode(self, tokenizer, text):

      # 前処理
      text = preprocessing_text(text)
      

      inputs = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          padding = 'max_length',
          truncation = True
      )
      return inputs

  def __getitem__(self, index):
    text = self.X[index]
    label = self.y[index]
    # userID = self.uid[index]
    ids = []
    mask = []
    inputs = self.encode(tokenizer=self.tokenizer, text=text)
    ids.append(torch.LongTensor(inputs['input_ids']))
    mask.append(torch.LongTensor(inputs['attention_mask']))

    return {
      'ids': ids,
      'mask': mask,
      'label': label,
      'text':text,
      # 'userID':userID
    }

In [81]:
# tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [82]:
X = df_train["text"].values
y = df_train["label"].values

In [83]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, train_size=0.75)

print(len(X_train))
print(len(y_train))
print(len(X_eval))
print(len(y_eval))

X_test = df_test["text"].values
y_test = df_test["label"].values
print(len(X_test))
print(len(y_test))

18750
18750
6250
6250
25000
25000


In [84]:
dataset_train = CreateDataset(X_train, y_train, tokenizer, max_len=max_len)
dataset_eval = CreateDataset(X_eval, y_eval, tokenizer, max_len=max_len)
dataset_test = CreateDataset(X_test, y_test, tokenizer, max_len=max_len)

print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

18750
6250
25000


In [85]:
dataset_train[1]

{'ids': [tensor([  101,  2119,  1293,  1108,   146,  6699,  1106,  1221,  1142,  1108,
           1103, 14908,  3919, 10626,  2188,  1182, 12153, 10041,   146,  1928,
            117,   146,  2788,  1142,  1165,  1115, 23569, 10220,  2799,  1146,
           1105,  1118,  1103,  1159,  1103,  1730,  2483,  1189,  1117,  2468,
            146,  1108, 23474,  9322,  2977, 22867,  7535, 18671,  1106,  1991,
            119,   119,   119,  1256,  1463,   146,  4819,  1115,  1461,   119,
           4753, 22774,  1116,  1110,  1164,   170,  1685, 12686, 12948,  1596,
           1873,  1150,  1110,  1107,  1103,  1965,  1104,  1217, 14647,  1174,
           1149,  1104,  1103,  2704,  1118,  1123,  1166,  9760,  1401,  1165,
           1152,   117,  1105,  1103,  1832,  1104,  1103,  1234,  1107,  1115,
           2440,  7605,   117,  1561, 17429,  7333,   119,   119,   119,  1332,
           1152,  6657,  1120,  1147,  1837,  1122,  2502,  1112,  2385,   170,
           3774,  1106,  1525,  1

In [86]:
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, pin_memory=True)
dataloader_eval = DataLoader(dataset_eval, batch_size=batch_size, shuffle=False, pin_memory=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, pin_memory=True)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dataloader_train, "val": dataloader_eval}

In [87]:
tmp = next(iter(dataloader_train))
print(tmp["ids"][0].size())
print(tmp["label"])
tmp["ids"][0][0]


torch.Size([16, 128])
tensor([1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])


tensor([  101,  2711,  1217,  2385,  1677,  2856,  1121,  1139, 11471,   117,
          146,  1108, 12678,  7351,  1118,  8166, 27400,  1162,  8166,   119,
          146, 12765,  1122,  1136,  3650,  1277,  1164,  1122,   117,  1133,
          146,  7588,  2637,  1122,  1106,  1129,   170,  8317,  3959,  2168,
         1273,  1107,  1103,  2530,  3475,  3462,  2168,  3904,   117,  1104,
         1134,   146,  1821,   170,  6970,  5442,   119,   146,  2207,  1146,
         2033,  1380,  3665,  1472,   117,  1134,  1110,  1136,  1120,  1155,
          170,  2213,  1645,   119,  1799,  1103,  1273,  1180,  1129,  5667,
         1112,  1216,   117,  1105,  1175,  1110,  5397,  1199,  1363,  2168,
         1105,  1289,  1106,  1289,  4127,  4429,  1107,  1103,  1273,   117,
         1122,  1110,  5397,  1136,  1103,  2425,  2817,   119,  2098,  2650,
         1132, 13157,  1193,  1167,  1696,  1106,  1103,  1273,  1190,  1157,
         9718,   117,   170,  1897, 14124,  1645,  1107,   102])

In [173]:
# model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", output_attentions=True, output_hidden_states=True)
model = BertModel.from_pretrained("bert-base-cased", output_attentions=True, output_hidden_states=True)

In [175]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにPosiNegaの2クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにクラス予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は2クラス
        self.cls = nn.Linear(in_features=768, out_features=2)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

        # カウント
        self.count = 0

    def forward(self, input_ids, attention_show_flg):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output
        # print(result)
        if self.count==0:
          print(f"Size of attentions: {result.attentions[-1].size()}")
          print(f"Size of sequence_output: {result[0].size()}")
          print(f"Size of pooled_output: {result[1].size()}")

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語(cls)の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        self.count += 1

        if attention_show_flg:
          return output, result.attentions[-1]
        else:
          return output

In [176]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')

ネットワーク設定完了


In [177]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True

In [178]:
# 最適化手法の設定
import torch.optim as optim


# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

In [179]:
# モデルを学習させる関数を作成

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch["ids"][0].to(device)  # 文章
                labels = batch["label"].to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs, attention_show_flg=False)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net

In [180]:
# 学習・検証を実行する。
num_epochs = 3
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
Size of attentions: torch.Size([16, 12, 128, 128])
Size of sequence_output: torch.Size([16, 128, 768])
Size of pooled_output: torch.Size([16, 768])
イテレーション 10 || Loss: 0.6524 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 20 || Loss: 0.6329 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 30 || Loss: 0.6880 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 40 || Loss: 0.6838 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 50 || Loss: 0.5832 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 60 || Loss: 0.6440 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 70 || Loss: 0.5694 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 80 || Loss: 0.5838 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 90 || Loss: 0.4903 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 100 || Loss: 0.4939 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 110 || Loss: 0.4288 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 120 || Loss: 0.6025 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 130 || Loss: 0.4541 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 140 || Loss: 0.3165 || 10iter.

In [181]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

# モデル評価用データ
labels_all = []
preds_all = []

for batch in tqdm(dataloader_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    inputs = batch["ids"][0].to(device)  # 文章
    labels = batch["label"].to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs, attention_show_flg=False)


        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

        # f1計算用
        labels_all.extend(batch["label"].to('cpu').detach().numpy())
        preds_all.extend(preds.to('cpu').detach().numpy())

# 正解率
epoch_acc = epoch_corrects.double() / len(dataloader_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dataloader_test.dataset), epoch_acc))

100%|██████████| 1563/1563 [03:07<00:00,  8.33it/s]

テストデータ25000個での正解率：0.8612


In [183]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print(f"正解率: {accuracy_score(labels_all, preds_all):.3f}")
print(f"適合率: {precision_score(labels_all, preds_all):.3f}")
print(f"再現率: {recall_score(labels_all, preds_all):.3f}")
print(f"F1: {f1_score(labels_all, preds_all):.3f}")

正解率: 0.861
適合率: 0.827
再現率: 0.913
F1: 0.868


# Attentionの可視化

In [184]:
# BertForIMDbで処理

# ミニバッチの用意
batch = next(iter(dataloader_test))

# GPUが使えるならGPUにデータを送る
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
inputs = batch["ids"][0].to(device)  # 文章
labels = batch["label"].to(device)  # ラベル

outputs, attention_probs = net_trained(inputs, attention_show_flg=True)

_, preds = torch.max(outputs, 1)  # ラベルを予測


In [185]:
attention_probs.size()

torch.Size([16, 12, 128, 128])

In [186]:
# HTMLを作成する関数を実装


def highlight(word, attn):
    "Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights):
    "HTMLデータを作成する"

    # indexの結果を抽出
    sentence = batch["ids"][0][index]  # 文章
    label = batch["label"][index]  # ラベル
    pred = preds[index]  # 予測

    # ラベルと予測結果を文字に置き換え
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 表示用のHTMLを作成する
    html = '正解ラベル：{}<br>推論ラベル：{}<br><br>'.format(label_str, pred_str)

    # Self-Attentionの重みを可視化。Multi-Headが12個なので、12種類のアテンションが存在
    for i in range(12):

        # indexのAttentionを抽出と規格化
        # 0単語目[CLS]の、i番目のMulti-Head Attentionを取り出す
        # indexはミニバッチの何個目のデータかをしめす
        attens = normlized_weights[index, i, 0, :]
        attens /= attens.max()

        html += '[BERTのAttentionを可視化_' + str(i+1) + ']<br>'
        for word, attn in zip(sentence, attens):

            # 単語が[SEP]の場合は文章が終わりなのでbreak
            if tokenizer.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
                break

            # 関数highlightで色をつける、関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
            html += highlight(tokenizer.convert_ids_to_tokens(
                [word.numpy().tolist()])[0], attn)
        html += "<br><br>"

    # 12種類のAttentionの平均を求める。最大値で規格化
    all_attens = attens*0  # all_attensという変数を作成する
    for i in range(12):
        attens += normlized_weights[index, i, 0, :]
    attens /= attens.max()

    html += '[BERTのAttentionを可視化_ALL]<br>'
    for word, attn in zip(sentence, attens):

        # 単語が[SEP]の場合は文章が終わりなのでbreak
        if tokenizer.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
            break

        # 関数highlightで色をつける、関数tokenizer_bert.convert_ids_to_tokensでIDを単語に戻す
        html += highlight(tokenizer.convert_ids_to_tokens(
            [word.numpy().tolist()])[0], attn)
    html += "<br><br>"

    return html


In [187]:
from IPython.display import HTML

index = 4  # 出力させたいデータ
html_output = mk_html(index, batch, preds, attention_probs)  # HTML作成
HTML(html_output)  # HTML形式で出力
